# IMPORTAR LIBRERIAS

In [1]:
!pip install tensorflow

In [2]:
import pandas as pd

In [8]:
X = pd.read_csv('../raw_data/X_per_h.csv', index_col = 0)
X.head()

/home/rociocisterna/.pyenv/versions/3.8.12/envs/modelo_energetico/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,ac_t_conf,ac_t_red,ac_mask,heat_t_conf,heat_t_red,heat_mask,ventilation_t,ventilation_vol,ventilation_mask,occupancy,...,facade_1_thickness_2,facade_1_window_area_percent,facade_2_thickness_2,facade_2_window_area_percent,facade_3_thickness_2,facade_3_window_area_percent,facade_4_thickness_2,facade_4_window_area_percent,roof_thickness_2,ground_thickness_2
0,24.0,25.5,0.0,21.0,19.0,0.0,20.0,1.0,0.0,0.0,...,0.15,45.0,0.06,45.0,0.11,40.0,0.06,50.0,0.11,20.0
1,24.0,25.5,0.0,21.0,19.0,0.0,20.0,1.0,0.0,0.0,...,0.15,45.0,0.06,45.0,0.11,40.0,0.06,50.0,0.11,20.0
2,24.0,25.5,0.0,21.0,19.0,0.0,20.0,1.0,0.0,0.0,...,0.15,45.0,0.06,45.0,0.11,40.0,0.06,50.0,0.11,20.0
3,24.0,25.5,0.0,21.0,19.0,0.0,20.0,1.0,0.0,0.0,...,0.15,45.0,0.06,45.0,0.11,40.0,0.06,50.0,0.11,20.0
4,24.0,25.5,0.0,21.0,19.0,0.0,20.0,1.0,0.0,0.0,...,0.15,45.0,0.06,45.0,0.11,40.0,0.06,50.0,0.11,20.0


In [9]:
from sklearn.model_selection import train_test_split


y = pd.read_csv('../raw_data/y_per_h.csv', index_col = 0)

y.head()

/home/rociocisterna/.pyenv/versions/3.8.12/envs/modelo_energetico/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Q_AC_OFFICE,Q_HEAT_OFFICE,Q_PEOPLE,Q_EQP,Q_LIGHT,Q_AHU_C,Q_AHU_H
0,0.0,0.0,0.0,0.0000,0.00,0.0,0.0
1,0.0,0.0,0.0,1.4718,0.21,0.0,0.0
2,0.0,0.0,0.0,1.4718,0.21,0.0,0.0
3,0.0,0.0,0.0,1.4718,0.21,0.0,0.0
4,0.0,0.0,0.0,1.4718,0.21,0.0,0.0


In [10]:
print(X.shape)
print(y.shape)

(5040000, 35)
(5040000, 7)


In [11]:
y['Q'] =y.iloc[:,0:7].sum(axis=1)
y.drop(columns=['Q_AC_OFFICE', 'Q_HEAT_OFFICE', 'Q_PEOPLE', 'Q_EQP',  'Q_LIGHT', 'Q_AHU_C', 'Q_AHU_H'],inplace=True)
y

,Q
0,0.0000
1,1.6818
2,1.6818
3,1.6818
4,1.6818
...,...
667,7.7406
668,7.7406
669,7.7406
670,7.7406


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

s_scaler = StandardScaler()

X_train_scaled = pd.DataFrame(s_scaler.fit_transform(X_train))
X_test_scaled = pd.DataFrame(s_scaler.transform(X_test))
y_train_scaled = pd.DataFrame(s_scaler.fit_transform(y_train))
y_test_scaled = pd.DataFrame(s_scaler.transform(y_test))

In [15]:
X_train_scaled.columns = X_train.columns
X_test_scaled.columns = X_test.columns
y_train_scaled.columns = y_train.columns
y_test_scaled.columns = y_test.columns

In [16]:
X_train_scaled.shape

(3376800, 35)

In [17]:
y_train_scaled.columns

Index(['Q'], dtype='object')

In [18]:
y_train_scaled.shape

(3376800, 1)

# RED 

In [19]:
from tensorflow.keras import *

2021-10-20 08:25:41.771783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-20 08:25:41.771820: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

In [21]:
def det_coeff(y_test_scaled, y_pred):
    u = K.sum(K.square(y_test_scaled - y_pred))
    v = K.sum(K.square(y_test_scaled - K.mean(y_test_scaled)))
    return K.ones_like(v) - (u / v)

In [22]:
def initialize_model():
    
    model = Sequential()
    model.add(layers.Dense(100, activation='relu',input_dim=35))    
    model.add(layers.Dropout(0.2))   
    model.add(layers.Dense(25, activation='relu'))
    
    model.add(layers.Dense(20, activation='relu'))
        
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='linear'))
    
    
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=[det_coeff])
             
    
    return model

model = initialize_model()

es = EarlyStopping(patience=25, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train_scaled,
                    validation_split=0.3,
                    #validation_data=(X_test_scaled, y_test),
                    epochs=500, #play with this until your validation loss overfit
                    batch_size=32, # Keep batch size to 16 today
                    verbose=0,
                    callbacks=[es])

2021-10-20 08:25:53.383658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-20 08:25:53.383703: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-20 08:25:53.383719: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-5UTQUOQ): /proc/driver/nvidia/version does not exist
2021-10-20 08:25:53.384009: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-20 08:25:54.122679: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 661

KeyboardInterrupt: 

In [ ]:
model.summary();

In [ ]:
results = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f'The r_square on the test set is of {results[1]:.2f}')

In [ ]:
results

In [46]:
y_pred= model.predict(X_test_scaled)

In [ ]:
data_df = pd.DataFrame(y_pred)

In [ ]:
data_df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,.5))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('Model mean_absolute_error')
    ax[1].set_ylabel('mean_absolute_error')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,0.5))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [ ]:
plot_loss_accuracy(history)